<p style="color:gray;
          text-align:center;
          font-size:24px;">
Inicialização de Bibliotecas, Escolha de Modelos e Leitura do Dataset Principal</p>


In [ ]:
#Inicialização de Bibliotecas, Escolha de Modelos e Leitura do Dataset Principal

import os
import ollama
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from statsmodels.stats.contingency_tables import cochrans_q,mcnemar
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report

models_names = ['all-minilm','gemma:7b','llama3:8b','mxbai-embed-large','phi3','phi3:14b','qwen2:7b','snowflake-arctic-embed2']
folder_models_names = [model.replace(':','_') for model in models_names]

try:
    os.makedirs(fr'Data')
except FileExistsError:
    pass
try:
    os.makedirs(fr'Results')
except FileExistsError:
    pass
try:
    os.rename(r'300-noticias-v2-filtradas.csv',r'Data\300-noticias-v2-filtradas.csv')
except FileNotFoundError:
    pass

seed = 7

data_frame = pd.read_csv(r'Data\300-noticias-v2-filtradas.csv')
data_frame

<p style="color:gray;
          text-align:center;
          font-size:24px;">
          Criação de Embedding</p>


In [ ]:
#Criação de Embedding

for model_name in models_names:
    try:
        os.makedirs(fr'Data\{model_name.replace(':','_')}')
    except FileExistsError:
        pass

    len_model = len(((ollama.embed(model=model_name,input='Test'))['embeddings'])[0])

    vector = np.empty((300,len_model),dtype=np.float32)
    label = np.empty((300,1),dtype='U5')

    for index,rotulo in enumerate(data_frame['Rótulo']):
        label[index] = rotulo

    for index,news in enumerate(data_frame['Texto_Original']):
        embedding_valor = ollama.embed(model=model_name,input=news)
        vector[index] = embedding_valor['embeddings'][0]
        labelad_vector = np.concatenate((label,vector),axis=1)
    labelad_df = pd.DataFrame(labelad_vector)
    labelad_df = labelad_df.rename(columns={labelad_df.columns[0]: 'Rótulo'})
    labelad_df.to_csv(path_or_buf= fr'Data\{model_name.replace(':','_')}\Embembedding.csv',
                  header=True,
                  index=False,
                  encoding='utf-8')

<p style="color:gray;
          text-align:center;
          font-size:24px;">
Separação de Dados</p>


In [ ]:
#Separação de Dados

for model_name in folder_models_names:
    np.random.seed(seed)

    model_df = np.array(pd.read_csv(fr'Data\{model_name}\Embembedding.csv'))
    k_vector = np.empty((model_df.shape[0]//10,model_df.shape[1]-1),dtype=np.float32)
    k_label = np.empty((model_df.shape[0]//10,1),'U5')
    k_fold = np.empty(model_df.shape,dtype=object)

    # O laço for abaixo cria 9 grupos de 30 noticias balanceados com metade 'real' metade 'falso'
    # O ultimo grupo é o resto das noticias que não foram distribuidas onde 17 são 'real' e 13 'falso'

    for j in range(9):

        for i in range(k_fold.shape[0]//20):
            
            passkey = 0
            
            while passkey != 'real':
                random_value = np.random.randint(0,model_df.shape[0])
                k_vector[2*i] = model_df[random_value,1:]
                k_label[2*i] = model_df[random_value,0]
                passkey = model_df[random_value,0]
            
            model_df = np.delete(model_df,random_value,axis=0)

            while passkey != 'falso':
                random_value = np.random.randint(0,model_df.shape[0])
                k_vector[2*i + 1] = model_df[random_value,1:]
                k_label[2*i +1] = model_df[random_value,0]
                passkey = model_df[random_value,0]
            
            model_df = np.delete(model_df,random_value,axis=0)
        
        k_labelad = np.concatenate((k_label,k_vector),axis=1)
        np.random.shuffle(k_labelad)

        k_fold[j*30:(j+1)*30,:] = k_labelad
    
    k_fold[k_fold.shape[0] - model_df.shape[0]:k_fold.shape[0],:] = model_df
    
    k_fold = pd.DataFrame(k_fold)

    k_fold.to_csv(path_or_buf= fr'Data\{model_name}\K_fold.csv',
                header=True,
                index=False,
                encoding='utf-8')

<p style="color:gray;
          text-align:center;
          font-size:24px;">
          Treinamento e Classificação</p>


In [ ]:
#Treinamento e Classificação

forest_seed = np.random.randint(0,100,size=10)

for model_name in folder_models_names:
    k_fold_orig = np.array(pd.read_csv(fr'Data\{model_name}\K_fold.csv'))

    k_fold_labels = np.empty((300,1),dtype='U5')
    
    for i in range(10):
        k_fold = k_fold_orig.copy()

        x_test = k_fold[30*i:30*(i+1),1:]
        y_test = k_fold[30*i:30*(i+1),0]
        
        k_fold = np.delete(k_fold,np.arange(30*i,30*(i+1)),axis=0)
        
        x_train = k_fold[:,1:]
        y_train = k_fold[:,0]
    
        RFC = (RandomForestClassifier(max_features=None,random_state=forest_seed[i])).fit(x_train,y_train)
    
        y_pred = ((RFC).predict(x_test)).reshape(30,1)
        k_fold_labels[30*i:30*(i+1),:] = y_pred
    
    k_fold_labels = pd.DataFrame(k_fold_labels)

    k_fold_labels.to_csv(path_or_buf= fr'Data\{model_name}\Prediction.csv',
                header=True,
                index=False,
                encoding='utf-8')

<p style="color:gray;
          text-align:center;
          font-size:24px;">
          Matrizes de Métricas</p>


In [ ]:
#Matrizes de Métricas

k_n = (np.array([f'K-{i+1}' for i in range(10)] + ['Means'])).reshape(11,1)
mean_matrix = np.empty(shape=(len(folder_models_names),7),dtype='U20')

for model_mean,model_name in enumerate(folder_models_names):

    metrics_matrix = np.empty(shape=(10,7))

    vector_pred = np.array(pd.read_csv(fr'Data\{model_name}\Prediction.csv'))
    vector_true = np.array(pd.read_csv(fr'Data\{model_name}\K_fold.csv'))[:,0]

    for i in range(10):
        
        y_pred = vector_pred[30*i:30*(i+1)]
        y_true = vector_true[30*i:30*(i+1)]

        matrix = confusion_matrix(y_true=y_true,y_pred=y_pred)

        
        fake_precision = matrix[0,0] / (matrix[0,0] + matrix[1,0])
        fake_recall = matrix[0,0] / (matrix[0,0] + matrix[0,1])
        fake_f1 = (2*fake_precision*fake_recall)/(fake_precision + fake_recall)

        true_precision = matrix[1,1] / (matrix[1,1] + matrix[0,1])
        true_recall = matrix[1,1] / (matrix[1,1] + matrix[1,0])
        true_f1 = (2*true_precision*true_recall) / (true_precision + true_recall)

        f1_macro = (fake_f1 + true_f1)/2 

        matrix_line = np.round(np.array([fake_precision,fake_recall,fake_f1,true_precision,true_recall,true_f1,f1_macro]).reshape(1,-1),2)

        metrics_matrix[i] = matrix_line

    pd.DataFrame(metrics_matrix).to_csv(path_or_buf=fr'Data\{model_name}\K_fold_Metrics.csv',
                            index=False,
                            header=True,
                            encoding='utf-8')

    std = np.array(np.std(metrics_matrix,axis=0)).reshape(1,metrics_matrix.shape[1])
    error_margin = (np.round(np.array([1.96*std_value/(metrics_matrix.shape[0]**(1/2)) for std_value in std]),2)*100).astype(np.uint8)
    mean_line = (np.round(np.array(np.mean(metrics_matrix,axis=0)).reshape(1,metrics_matrix.shape[1]),2)*100).astype(np.uint8)
    mean_line = np.char.add(np.char.add(mean_line.astype('U5'),'% ± '),error_margin.astype('U5'))
    mean_matrix[model_mean] = mean_line

    metrics_matrix = (np.round(metrics_matrix,2)*100).astype(np.uint8)

    metrics_data_frame = np.concatenate((metrics_matrix,mean_line),axis=0)
    metrics_data_frame = np.concatenate((k_n,metrics_data_frame),axis=1)
    metrics_labels = np.array([['K_Group',' ','Fake','  ','   ','True','    ','f₁ - Macros'],
                               [' ','p','r','f₁','p','r','f₁','  ']])
    metrics_data_frame = np.concatenate((metrics_labels,metrics_data_frame),axis=0)
    metrics_data_frame[2:,1:] = np.char.add(metrics_data_frame,'%')[2:,1:]
    metrics_data_frame = pd.DataFrame(metrics_data_frame)

    metrics_data_frame.to_csv(path_or_buf=fr'Results\{model_name}\Metric_Matrix.csv',
                            index=False,
                            header=False,
                            encoding='utf-8')

    metrics_labels[0,0] ='Models'
    metrics_data_frame = np.concatenate((np.array(models_names).reshape(len(models_names),1),mean_matrix),axis=1)
    metrics_data_frame = np.concatenate((metrics_labels,metrics_data_frame),axis=0)
    metrics_data_frame[2:,1:] = np.char.add(metrics_data_frame[2:,1:],'% ')
    metrics_data_frame = pd.DataFrame(metrics_data_frame)

    metrics_data_frame.to_csv(path_or_buf=r'Results\Means_Matrix.csv',
                            index=False,
                            header=False,
                            encoding='utf-8')
metrics_data_frame

<p style="color:gray;
          text-align:center;
          font-size:24px;">
          Testes Estatísticos </p>

In [91]:
#Testes Estatísticos

score_matrix = np.ones(shape=(300,len(folder_models_names)),dtype=np.int8)

for model_column,model_name in enumerate(folder_models_names):

    label = np.array(pd.read_csv(fr'Data\{model_name}\K_fold.csv'))[:,0]
    predictions = np.array(pd.read_csv(fr'Data\{model_name}\Prediction.csv'))[:,0]
    error_list = []

    for i in range(300):
        if predictions[i] != label[i]:
            error_list.append(i)
    
    for error_index in error_list:
        score_matrix[error_index,model_column] = 0

score_matrix = pd.DataFrame(score_matrix)
score_matrix.columns = folder_models_names
score_matrix.to_csv(path_or_buf=fr'Results\Score_Matrix.csv',
                            index=False,
                            header=True,
                            encoding='utf-8')

# Teste de Q-Cochrans para ver se há diferença entre as predições dos modelos (p-valor<0.05)

alpha = 0.05

cochrans_q_result = cochrans_q(score_matrix.values)

cochrans_q_vector = np.array([['Estatística',round(cochrans_q_result.statistic,4)],
                           ['p-valor',round(cochrans_q_result.pvalue,4)],
                           ['Significativo',cochrans_q_result.pvalue < alpha]])

cochrans_q_df = pd.DataFrame({'Metricas':cochrans_q_vector[:,0],'Valores':cochrans_q_vector[:,1]})

#Se houver diferenças vamos realizar o teste de Mcnemmar entre todos os modelos dois a dois
#Para evitar o erro de Falso positivo vamos usar a correção de Bonferroni

combinations =[]

for i in range(len(folder_models_names)):
    for j in range(i+1,len(folder_models_names)):
        combinations.append((folder_models_names[i],folder_models_names[j]))


alpha_bonferroni = alpha/len(combinations)

mcnemar_results = []

for model_a,model_b in combinations:
    
    cross_tab = confusion_matrix(score_matrix[model_a],score_matrix[model_b])

    mcnemar_result = mcnemar(cross_tab,exact=False,correction=True)

    mcnemar_results.append({
        'Modelo A': model_a,
        'Modelo B': model_b,
        'Estatística': round(mcnemar_result.statistic,4),
        'p-valor': round(mcnemar_result.pvalue,4),
        'Significativo (Bonferroni)': mcnemar_result.pvalue < alpha_bonferroni
    })

mcnemar_df = pd.DataFrame(mcnemar_results)

if (True not in np.array(mcnemar_df.iloc[:,4])) and (cochrans_q_df.iloc[2,1] != True):
    print("\033[1;32mNão há Diferenças estatísticas entre os modelos\033[0m")
else:
    print("\033[1;31mHá Diferenças estatísticas entre os modelos\033[0m")

score_ranking = pd.DataFrame((score_matrix.sum()).sort_values(ascending=False))
score_ranking.columns =['Score']
score_ranking['Score %'] = np.char.add((score_ranking.values/3).astype('U4'),' %')

display(cochrans_q_df.style.set_caption("<span style='font-size: 16px; font-weight: bold;'>Teste de Q-Cochrans</span>"))
display(mcnemar_df.style.set_caption("<span style='font-size: 20px; font-weight: bold;'>Teste de McNemar</span>"))
display(score_ranking.style.set_caption("<span style='font-size: 16px; font-weight: bold;'>Score Ranking</span>"))

Há Diferenças estatísticas entre os modelos


,Metricas,Valores
0,Estatística,14.4599
1,p-valor,0.0436
2,Significativo,True


,Modelo A,Modelo B,Estatística,p-valor,Significativo (Bonferroni)
0,all-minilm,gemma_7b,0.735300,0.391200,False
1,all-minilm,llama3_8b,2.782600,0.095300,False
2,all-minilm,mxbai-embed-large,0.000000,1.000000,False
3,all-minilm,phi3,0.000000,1.000000,False
4,all-minilm,phi3_14b,1.440000,0.230100,False
5,all-minilm,qwen2_7b,0.160000,0.689200,False
6,all-minilm,snowflake-arctic-embed2,0.833300,0.361300,False
7,gemma_7b,llama3_8b,11.529400,0.000700,True
8,gemma_7b,mxbai-embed-large,0.484800,0.486200,False
9,gemma_7b,phi3,0.516100,0.472500,False


,Score,Score %
llama3_8b,291,97.0 %
phi3_14b,289,96.3 %
snowflake-arctic-embed2,288,96.0 %
qwen2_7b,285,95.0 %
all-minilm,282,94.0 %
mxbai-embed-large,281,93.6 %
phi3,281,93.6 %
gemma_7b,276,92.0 %


<p style="color:gray;
          text-align:center;
          font-size:24px;">
          Gráficos</p>


In [ ]:
#Gráficos

boolean_map = np.vectorize(lambda z: z == 'real')

for model_name in folder_models_names:

    try:
        os.makedirs(fr'Results\{model_name}')
    except FileExistsError:
        pass

    vector =np.array((pd.read_csv(fr'Data\{model_name}\Embembedding.csv')).iloc[:,1:])
    label = np.array((pd.read_csv(fr'Data\{model_name}\Embembedding.csv')).iloc[:,0])

    tsne_2d = TSNE(n_components=2,random_state=seed).fit_transform(vector)
    tsne_3d = TSNE(n_components=3,random_state=seed).fit_transform(vector)
    tsne_3d_true = tsne_3d[boolean_map(label)==True]
    tsne_3d_false = tsne_3d[boolean_map(label)==False]
    

    plt.figure(figsize=(12, 8))

    plt.scatter(tsne_2d[:,0],
                tsne_2d[:,1], 
                c=boolean_map(label),
                cmap='coolwarm',
                alpha=1,
                edgecolors='w',
                s=100)
    plt.title(f'{model_name}',fontsize = 32)

    plt.savefig(fr'Results\{model_name}\2d.png', 
                   dpi=300,
                   bbox_inches='tight',
                   facecolor='white')

    plt.show()

    
    interactive_graph = go.Figure()

    interactive_graph.add_trace(go.Scatter3d(
        x=tsne_3d_true[:,0],
        y=tsne_3d_true[:,1],
        z=tsne_3d_true[:,2],
        mode='markers',
        marker=dict(size=4, color='blue', opacity=1),
        name='Real'
    ))

    interactive_graph.add_trace(go.Scatter3d(
        x=tsne_3d_false[:,0],
        y=tsne_3d_false[:,1],
        z=tsne_3d_false[:,2],
        mode='markers',
        marker=dict(size=4, color='red', opacity=1),
        name='Falso'
    ))


    interactive_graph.add_trace(go.Scatter3d(
    x=[-100,100],
    y=[0, 0],
    z=[0, 0],
    mode='lines',
    line=dict(color='black', width=5),
    name='Eixo X'
    ))

    interactive_graph.add_trace(go.Scatter3d(
        x=[0, 0],
        y=[-100,100],
        z=[0, 0],
        mode='lines',
        line=dict(color='black', width=5),
        name='Eixo Y'
    ))

    interactive_graph.add_trace(go.Scatter3d(
        x=[0, 0],
        y=[0, 0],
        z=[-100, 100],
        mode='lines',
        line=dict(color='black', width=5),
        name='Eixo Z'
    ))

    interactive_graph.update_layout(
    
    title={'text':f'<b>{model_name}</b>',
           'x': 0.5,
           'y': 0.95},
             title_font_size=32,
    
    scene=dict(
        xaxis=dict(visible=False),
        
        yaxis=dict(visible=False),

        zaxis=dict(visible=False)),
    paper_bgcolor='white',
    plot_bgcolor='white')

    interactive_graph.write_html(fr'Results\{model_name}\3d.html')

    interactive_graph.show()

    vector_pred = np.array(pd.read_csv(fr'Data\{model_name}\Prediction.csv'))
    vector_true = np.array(pd.read_csv(fr'Data\{model_name}\K_fold.csv'))[:,0]

    matrix = confusion_matrix(y_true=vector_true,y_pred=vector_pred)

    sns.heatmap(matrix,annot=True,fmt='d',cmap='Blues')
    plt.xlabel('Falso           (Previsto)           Real')
    plt.ylabel('Real           (Rótulo)           Falso')
    plt.title(f'{model_name}',fontsize = 24)
    plt.savefig(fr'Results\{model_name}\Confusion_Matrix.png', 
                dpi=300,
                bbox_inches='tight',
                facecolor='white')
    plt.show()

metrics_tensor = np.load(r'Results\Metrics_Tensor.npy')

try:
    os.makedirs(fr'Results\Boxplots')
except FileExistsError:
    pass

metrics_header = np.concatenate(('Fake-' + metrics_labels[1,1:4],'True-' + metrics_labels[1,4:-1]),axis = 0)
metrics_header = np.append(metrics_header,metrics_labels[0,7])

for i in range(len(metrics_header)):
    plt.figure(figsize=(8, 6))
    plt.boxplot(metrics_tensor[:,:,i].T,tick_labels=folder_models_names)
    plt.xticks(rotation=45, fontsize=8)
    plt.title(f'Boxplot de {metrics_header[i]}')
    plt.xlabel('Modelos')
    plt.ylabel('Valor')
    plt.grid(False)
    plt.savefig(fr'Results\Boxplots\Boxplot_{metrics_header[i]}')
    plt.show()
